**Diplomado en Inteligencia Artificial y Aprendizaje Profundo**

# Modelo logístico de predicción en tf.keras

##  Autores

1. Alvaro Mauricio Montenegro Díaz, ammontenegrod@unal.edu.co
2. Daniel Mauricio Montenegro Reyes, dextronomo@gmail.com 
3. Oleg Jarma, ojarmam@unal.edu.co
4. Maria del Pilar Montenegro, pmontenegro88@gmail.com

## Contenido

* [Introducción](#Introducción)
* [Importa módulos](#Importa-módulos)
* [Funciones de activación](#Funciones-de-activación)
* [Regularizadores ](#Regularizadores)
* [El conjunto de datos cáncer de seno Wisconsin](#El-conjunto-de-datos-cáncer-de-seno-Wisconsin)
* [Lectura de datos](#Lectura-de-datos)
* [Preprocesamiento](#Preprocesamiento)
* [Crea el modelo Sequential modo 2](#Crea-el-modelo-Sequential-modo-2)
* [Compila](#Compila)
* [Entrenamiento](#Entrenamiento)
* [Predicciones](#Predicciones)
* [Matriz de confusión](#Matriz-de-confusión)
* [Evaluación del modelo](#Evaluación-del-modelo)


## Introducción

En esta lección construimos un modelo predictivo de regresión logística usando los datos de cáncer de la Universidad de Wisconsin.

Usaremos la funciones de activación *relu* y *sigmoid* y el regularizador *Dropout*. Adicionalmente introducimos el  modo 2 de escribir un modelo *Sequiential*.

Finalmente se construye una matriz de confusión para el problema.

## Importa módulos

In [ ]:
try:
  %tensorflow_version 2.x
except Exception:
  pass

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
#
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
#
from tensorflow.keras.models import Sequential
#
from tensorflow.keras.layers import Dense, Dropout
#
from tensorflow.keras.utils import plot_model
#
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
#
print(tf.__version__)

## Funciones de activación

### Logística o sigmoide

Dada la salida del sumador digamos $y=\mathbf{w}'\mathbf{x} +b$, la función de activación *sigmoid* esta definida por

$$
\text{sigmoid}(y) = \frac{1}{1+\exp(-y)}
$$

Entonces, esta función de activación transforma cualquier número real a un número en el intervalo $(0,1)$, de tal manera que puede ser interpretado como una probabilidad. 

### Relu

Dada la salida del sumador digamos $y=\mathbf{w}'\mathbf{x} +b$, la función de activación *relu* esta definida por

$$
\text{relu}(y) = \begin{cases} &0, \text{ si } y\le 0,\\
 &y, \text{ en otro caso } \end{cases}
$$



## Regularizadores 

### Dropout

Usaremos el regularizador Dropout. [Dropout paper](https://jmlr.org/papers/v15/srivastava14a.html). Este regularizador deja por fuera del entrenamiento en cada epoch un porcentaje de las neuronas de la capa, escogidas de forma aleatoria en cada epoch. Por ejemplo *Dropout(0.1)* deja el 10% de las neuronas por fuera del entrenamiento en cada epoch. En el artículo muestran que en muchos casos este regularizador funciona mejor que los clásicos L1 y L2.

## El conjunto de datos cáncer de seno Wisconsin


Puede descargar los datos en [kaggle- cancer de seno-Wisconsin](https://www.kaggle.com/uciml/breast-cancer-wisconsin-data)


### Meta-información sobre los datos


1. ID number
2. Diagnosis (M = maligno, B = benigno) 

Se calculan diez características de valor real para cada núcleo celular:

  - radius (media de las distancias desde el centro a los puntos del perímetro) 
  - texture (desviación estándar de los valores de la escala de grises) 
  - perimeter
  - area
  - smoothness (variación local en las longitudes de los radios)
  - compactness (perímetro ^ 2 / área - 1.0) 
  - concavity (severidad de las porciones cóncavas del contorno) 
  - concave points (número de porciones cóncavas del contorno) 
  - symetry 
  - fractal dimension ("aproximación de la línea de costa"
 
La media, el error estándar y el "peor" o el mayor (la media de los tres valores más grandes) de estas características se calcularon para cada imagen, lo que resultó en 30 características. Por ejemplo, el campo 3 es Radio medio, el campo 13 es Radio SE, el campo 23 es Peor radio.

Todos los valores de las características se recodifican con cuatro dígitos significativos.

datos faltantes: ninguno

Distribución de clases: 357 benignos, 212 malignos




## Lectura de datos

### separa entrada (features) y salida 

In [ ]:
# Importing data
data = pd.read_csv('../Datos/breast_cancer/data.csv')
del data['Unnamed: 32']

## Preprocesamiento

In [ ]:
x = data.iloc[:,2:].values # extrae como tensores numpy
y = data.iloc[:,1].values

### Recodifica la variable objetivo

In [ ]:
labelencoder_y = LabelEncoder()
y = labelencoder_y.fit_transform(y)

### Divide los datos. Entrenamiento y test

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.1, random_state = 0)


In [ ]:
x_test.shape

### Normaliza los datos

In [ ]:
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

## Crea el modelo Sequential modo 2

In [ ]:
classifier = Sequential()
## Adiciona capas una por una
classifier.add(Dense(units=16, activation='relu', input_shape=(30,)))
# Adding dropout to prevent overfitting (regularización)
classifier.add(Dropout(0.1)) # 10% out in each epoc
classifier.add(Dense(units=16, activation='relu'))
# Adding dropout to prevent overfitting (regularización)
classifier.add(Dropout(0.1))
classifier.add(Dense(units=1, activation='sigmoid'))


## Compila

In [ ]:
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
classifier.summary()
plot_model(classifier, to_file='../Imagenes/cancer_seno.png', 
           show_shapes=True)

## Entrenamiento

In [ ]:
history = classifier.fit(x_train, y_train, batch_size=32, epochs=150,validation_split = 0.2)

## Predicciones

In [ ]:
# Predicting the Test set results
y_pred = classifier.predict(x_test)
#y_pred = (y_pred > 0.5)
y_pred[y_pred > 0.5] = 1
y_pred[y_pred <=0.5] = 0

## Matriz de confusión

In [ ]:
cm = confusion_matrix(y_test, y_pred)

In [ ]:
print("Our accuracy is {}%".format(((cm[0][0] + cm[1][1])/y_test.shape[0])*100))

In [ ]:
sns.heatmap(cm,annot=True)
plt.savefig('h.png')

## Evaluación del modelo

In [ ]:

def plot_metric(history, metric):
    train_metrics = history.history[metric]
    val_metrics = history.history['val_'+metric]
    epochs = range(1, len(train_metrics) + 1)
    plt.plot(epochs, train_metrics, 'bo--')
    plt.plot(epochs, val_metrics, 'ro-')
    plt.title('Entrenamiento y validación '+ metric)
    plt.xlabel("Epochs")
    plt.ylabel(metric)
    plt.legend(["train_"+metric, 'val_'+metric])
    plt.show()

In [ ]:
plot_metric(history, 'loss')

In [ ]:
plot_metric(history, 'accuracy')

[Ir al inicio](#Contenido)